In [ ]:
import overpy
import pandas as pd 
import requests
import json
import pymssql
from sqlalchemy import Integer, String, Float, DATETIME, create_engine

In [ ]:
# Load configuration from config/db_config.json
with open('../config/db_config.json', 'r') as f:
    db_config = json.load(f)

# Get database credentials
server = db_config['server']
database = db_config['database']
db_user = db_config['db_user']
db_password = db_config['db_password']

In [ ]:
# Connect to SQL Database
conn = pymssql.connect(server, db_user, db_password, database)

# Create connection string for SQLAlchemy
connection_string = f"mssql+pymssql://{db_user}:{db_password}@{server}/{database}"
engine = create_engine(connection_string)

In [ ]:
# SQL-Abfrage für die gewünschten Spalten
query = "SELECT DISTINCT id FROM OVRP_HikingRoutes"

# Daten aus der Azure SQL-Datenbank laden
relation_ids = pd.read_sql_query(query, con=engine)

# Ergebnis anzeigen
print(relation_ids)


In [ ]:
relation_ids = [12624412, 15527560, 15527560]  # Beispiel-IDs

In [ ]:
api = overpy.Overpass(url="http://overpass.osm.ch/api/interpreter")

timestamp_apicall = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")


# Leere Liste für die Ergebnisse
results = []

# Overpass-Abfragen iterativ durchführen
for relation_id in relation_ids:
    # Overpass-Abfrage
    
    query = f"""
    [out:json];
    relation["route"="hiking"](id:{relation_id});
    out ids;  // Gibt die Relation-ID aus
    way(r);
    out ids center tags;  // Gibt die Way-IDs und ihre Mittelpunkte aus
    """
    result = api.query(query)
    # Anfrage an die Overpass-API senden
   
    for way in result.ways:
            if way.center_lat and way.center_lon:
                # Daten als Dictionary speichern
                results.append({
                    "relation_id": relation_id,
                    "way_id": way.id,
                    "lat": way.center_lat,
                    "lon": way.center_lon,
                    "highway": way.tags.get("highway", None),
                    "surface": way.tags.get("surface", None),
                    "timestamp_apicall": timestamp_apicall
                })

# Convert lat and lon to numeric, timestamp to datetime



# Ergebnisse in einen DataFrame konvertieren
df_wege = pd.DataFrame(results)
df_wege['lat'] = pd.to_numeric(df_wege['lat'], errors='coerce')
df_wege['lon'] = pd.to_numeric(df_wege['lon'], errors='coerce')
df_wege['timestamp_apicall'] = pd.to_datetime(df_wege['timestamp_apicall'], errors='coerce')

# Ergebnis anzeigen
print(df_wege)




In [ ]:
# Create table if it doesn't exist
table_name = "OVRP_ways"
query = f"""
    IF OBJECT_ID(N'dbo.{table_name}', N'U') IS NULL
    BEGIN
        CREATE TABLE {table_name} (
            id                      INT         NOT NULL,
            id_relation             INT         NOT NULL,
            lat                     FLOAT       NOT NULL,
            lon                     FLOAT       NOT NULL,
            highway                 VARCHAR(255) NULL,
            surface                 VARCHAR(255) NULL,
            timestamp_apicall       DATETIME    NULL,
            PRIMARY KEY (id)
            
        );
    END
    """

conn = pymssql.connect(server, db_user, db_password, database)
cursor = conn.cursor()
cursor.execute(query)

conn.commit()
conn.close()

In [ ]:
# Create connection string for SQLAlchemy
connection_string = f"mssql+pymssql://{db_user}:{db_password}@{server}/{database}"
engine = create_engine(connection_string)

# Ingest data to tabledatabase table
df_wege.to_sql(table_name, con=engine, if_exists='replace', index=False)
print("DataFrame erfolgreich in die MSSQL-Datenbank geladen!")